# Test Autograd

## Librerías

In [ ]:
import autograd.numpy as np  # Thinly-wrapped numpy
from autograd import grad
from functools import partial
from bisect import bisect_right

## First Try

In [2]:
def present_value(cashflows: np.array, dfs: np.array) -> float:
    return np.dot(cashflows, dfs)

In [3]:
grad_pv = grad(present_value)

In [4]:
valor_tasa = .1
grad_pv(np.array([5.0, 100.0]), np.array(
    [(1+valor_tasa/2)**-1, (1+valor_tasa)**-1]))

array([0.95238095, 0.90909091])

In [5]:
(1+valor_tasa/2)**-1

0.9523809523809523

In [6]:
present_value(np.array([5, 100]), np.array(
    [(1+valor_tasa/2)**-1, (1+valor_tasa)**-1]))

95.67099567099567

In [7]:
pv = partial(present_value, np.array([5.0, 100.0]))

In [8]:
grad_pv2 = grad(pv)

In [9]:
pv(np.array([(1+valor_tasa/2)**-1, (1+valor_tasa)**-1]))

95.67099567099567

In [10]:
grad_pv2(np.array([(1+valor_tasa/2)**-1, (1+valor_tasa)**-1]))

array([  5., 100.])

In [11]:
def two(x, y):
    return x*y

In [12]:
grad_two = grad(two)

In [13]:
grad_two(1.0, 2.0)

2.0

## Más Estrucuturado

In [14]:
tenors = np.array([1.0, 365.0])
rates = np.array([.05, .1])

In [15]:
def lin_interpol(tenors, rates, tenor):
    if tenor >= tenors[len(tenors) - 1]:
        return rates[len(tenors) - 1]
    elif tenor <= tenors[0]:
        return rates[0]
    else:
        i = bisect_right(tenors, tenor) - 1
        m = (rates[i + 1] - rates[i]) / (tenors[i + 1] - tenors[i])
        return rates[i] + m * (tenor - tenors[i])

In [16]:
def df(rate, tenor):
    return (1 + rate)**(-tenor / 365.0) 

In [17]:
def present_value(interp, disc, cashflow, tenor, tenors, rates):
    rate = interp(tenors, rates, tenor)
    return df(rate, tenor) * cashflow

In [18]:
present_value(lin_interpol, df, 100.0, 189.0, tenors, rates)

96.28621040299818

In [19]:
drates = grad(present_value, argnum=5)

In [20]:
drates(lin_interpol, df, 100.0, 180.0, tenors, rates)

array([-22.51132432, -21.78122732])

In [21]:
10000*10*4*6/1000/60

40.0

### Bootstrapping (1 Sólo Fujo)

In [22]:
cashflow = 107.5
plazo = 365.0

**Problema:** encontrar la tasa a 365 días que haga que el valor presente de este flujo sea igual a 100.

In [23]:
pv = partial(present_value, lin_interpol, df)

In [24]:
pv(cashflow, plazo, tenors, rates)

97.72727272727272

In [25]:
g = grad(pv, argnum=3)

In [26]:
g(cashflow, plazo, tenors, rates)

array([  0.        , -88.84297521])

In [27]:
rates0 = np.array([.05, 0.0])

In [28]:
def solve(cashflow, tenor, obj, tenors, rates, pv):
    epsilon = .000001
    g = grad(pv, argnum=3)
    rates_ = np.array([rates[0], rates[1]])
    delta = 1
    while delta > epsilon:
        r1 = rates_[1] - (pv(cashflow, tenor, tenors, rates_) - obj) / g(cashflow, tenor, tenors, rates_)[1]
        delta = abs(r1 - rates_[1])
        if type(r1) is np.float64:
            rates_[1] = r1
        else:
            rates_[1] = r1._value
    return r1

In [29]:
solve(cashflow, plazo, 100.0, tenors, rates0, pv)

0.07500000000000007

In [30]:
rates0

array([0.05, 0.  ])

In [31]:
gsolve = grad(solve, argnum=0)

In [32]:
gsolve(cashflow, plazo, 100.0, tenors, rates0, pv)

0.009999999988773293

### Bootstrapping (Varios Flujos)

In [33]:
def kron(i, j):
    return int(i == j)

In [34]:
def dfs(rates, tenors):
    return np.array([df(z[0], z[1]) for z in zip(rates, tenors)])

In [35]:
rates = np.array([.01, .075])
dfs(rates, tenors)

array([0.99997274, 0.93023256])

In [36]:
def lin_interpols(tenors, rates, new_tenors):
    return np.array([lin_interpol(tenors, rates, t) for t in new_tenors])

In [37]:
lin_interpols(tenors, rates, [90.0, 180.0, 270.0])

array([0.02589286, 0.04196429, 0.05803571])

In [38]:
def fixed_rate_leg(nocional, tasa, num_cupones):
    return np.array([.5 * 365.0 * i for i in range(1, num_cupones + 1)]), \
        np.array([nocional * (kron(i, num_cupones) + tasa / 2.0)
                  for i in range(1, num_cupones + 1)])

In [39]:
fixed_rate_leg(100, .06, 2)

(array([182.5, 365. ]), array([  3., 103.]))

In [40]:
def present_value_2(interp, disc, cashflows_tenors, cashflows, curve_tenors, curve_rates):
    cashflow_rates = interp(curve_tenors, curve_rates, cashflows_tenors)
    return np.dot(dfs(cashflow_rates, cashflows_tenors), cashflows)

In [41]:
present_value_2(lin_interpols, dfs, fixed_rate_leg(100, .06, 2)[0],
               fixed_rate_leg(100, .06, 2)[1], tenors, rates)

98.75229196044188

In [42]:
def pv_fixed_leg(interp, disc, nocional, tasa, num_cupones, curve_tenors, curve_rates):
    plazos, flujos = fixed_rate_leg(nocional, tasa, num_cupones)
    return present_value_2(interp, disc, plazos, flujos, curve_tenors, curve_rates)

In [43]:
pv_fixed_leg(lin_interpols, dfs, 100.0, .06, 2, tenors, rates)

98.75229196044188

In [44]:
pv_2 = partial(pv_fixed_leg, lin_interpols, dfs)

In [45]:
pv_2(100.0, .06, 2, tenors, rates)

98.75229196044188

In [46]:
gpvleg = grad(pv_2, argnum=1)

In [47]:
gpvleg(100.0, .06, 2, tenors, rates)

95.4839357748065

In [66]:
def solve_2(nocional, tasa, num_cupones, obj, tenors, rates, pv):
    epsilon = .000001
    g = grad(pv, argnum=4)
    rates_ = np.array([r for r in rates] + [0.0,])
    print(rates_)
    which = len(rates_) - 1
    print(f'which: {which}')
    delta = 1
    while delta > epsilon:
        q = (pv(nocional, tasa, num_cupones, tenors, rates_) - obj)
        q /= g(nocional, tasa, num_cupones, tenors, rates_)[which]
        r1 = rates_[which] - q
        print(r1)
        delta = abs(r1 - rates_[which])
        print(f'delta: {delta}')
        if type(r1) is np.float64:
            rates_ = np.array([r for r in rates] + [r1,])
        else:
            rates_ = np.array([r for r in rates] + [r1._value,])
    return r1

In [67]:
rates0 = np.array([.05,])

In [68]:
r_sol = solve_2(100.0, .06, 2, 100.0, tenors, rates0, pv_2)
print(f'sol: {r_sol}')

[0.05 0.  ]
which: 1
0.057491813930758244
delta: 0.057491813930758244
0.06097025747500407
delta: 0.0034784435442458267
0.0609816831491982
delta: 1.1425674194126545e-05
0.06098168327166905
delta: 1.2247085290351123e-10
sol: 0.06098168327166905


In [69]:
rates_sol = np.array([rates0[0], r_sol])
pv_2(100.0, .06, 2, tenors, rates_sol)

100.00000000000003

In [70]:
gs2 = grad(solve_2, argnum=1)

In [71]:
gs2(100.0, .06, 2, 100.0, tenors, rates0, pv_2)

[0.05 0.  ]
which: 1
Autograd ArrayBox with value 0.057491813930758244
delta: Autograd ArrayBox with value 0.057491813930758244
Autograd ArrayBox with value 0.06097025747500407
delta: Autograd ArrayBox with value 0.0034784435442458267
Autograd ArrayBox with value 0.0609816831491982
delta: Autograd ArrayBox with value 1.1425674194126545e-05
Autograd ArrayBox with value 0.06098168327166905
delta: Autograd ArrayBox with value 1.2247085290351123e-10


1.0390997985892199

In [72]:
grad(solve_2, argnum=5)(100.0, .06, 2, 100.0, tenors, rates0, pv_2)

Autograd ArrayBox with value [0.05 0.  ]
which: 1
Autograd ArrayBox with value 0.057491813930758244
delta: Autograd ArrayBox with value 0.057491813930758244
Autograd ArrayBox with value 0.06097025747500407
delta: Autograd ArrayBox with value 0.0034784435442458267
Autograd ArrayBox with value 0.0609816831491982
delta: Autograd ArrayBox with value 1.1425674194126545e-05
Autograd ArrayBox with value 0.06098168327166905
delta: Autograd ArrayBox with value 1.2247085290351123e-10


array([-0.00752311])

In [73]:
tenors0 = [1.0, 365.0, 730.0]
rates0 = [0.05, r_sol]

In [74]:
solve_2(100.0, .062, 4, 100.0, tenors0, rates0, pv_2)

[0.05       0.06098168 0.        ]
which: 2
0.05760642144390602
delta: 0.05760642144390602
0.06303452556938739
delta: 0.005428104125481371
0.06307638835226625
delta: 4.186278287886258e-05
0.06307639080868392
delta: 2.456417666496513e-09


0.06307639080868392

In [76]:
grad(solve_2, argnum=1)(100.0, .062, 4, 100.0, tenors0, rates0, pv_2)

[0.05       0.06098168 0.        ]
which: 2
Autograd ArrayBox with value 0.05760642144390602
delta: Autograd ArrayBox with value 0.05760642144390602
Autograd ArrayBox with value 0.06303452556938739
delta: Autograd ArrayBox with value 0.005428104125481371
Autograd ArrayBox with value 0.06307638835226625
delta: Autograd ArrayBox with value 4.186278287886258e-05
Autograd ArrayBox with value 0.06307639080868392
delta: Autograd ArrayBox with value 2.456417666496513e-09


1.0696313573198202